# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [0]:
# The following code is for preparing to work on the Google colab. Run the below if working on Colab only

In [0]:
# Colab Code
# Load the Drive helper and mount
from google.colab import drive

In [0]:
# Colab Code
# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls

drive				      model_init_2018-10-1908_40_41.690301
model_init_2018-10-1905_12_35.980103  model_init_2018-10-1908_53_32.996976
model_init_2018-10-1907_01_27.105106  Project_data
model_init_2018-10-1908_02_41.840117  sample_data


In [0]:
# Colab Code to extract data to the colab workspace
!unzip '/content/drive/My Drive/NeuralNetwork/NN.zip' 

Archive:  /content/drive/My Drive/NeuralNetwork/NN.zip
   creating: Project_data/
   creating: Project_data/train/
  inflating: Project_data/train.csv  
   creating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00012.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00014.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00016.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00018.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00020.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00022.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_

In [0]:
!pip install imageio

    100% |████████████████████████████████| 3.3MB 9.5MB/s 
  Running setup.py bdist_wheel for imageio ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/e0/43/31/605de9372ceaf657f152d3d5e82f42cf265d81db8bbe63cde1
Successfully built imageio


In [0]:
!pip install scikit-image

In [0]:
# To check if the processor is GPU

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import datetime

We set the random seed so that the results don't vary drastically.

In [0]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [0]:
# Change the datapath accordingly

#train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
#val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
train_doc = np.random.permutation(open('/content/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/Project_data/val.csv').readlines())
#batch_size = # choose the batch size
batch_size = 30
batch_data = np.zeros((batch_size,12,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output


In [0]:
print(len(train_doc))

200


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [0]:
import skimage.transform
from skimage import io

# from sklearn.preprocessing import scale

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
#     img_idx = #create a list of image numbers you want to use for a particular video
    img_idx = list(range(10, 22))
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t) // batch_size
        for batch in range(num_batches): # we iterate over the number of batches
#             batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size, 12, 100, 100, 3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = io.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]) #.astype(np.float32)
                    
                    image_resized = skimage.transform.resize(image, (100, 100))#, anti_aliasing=True)
                    
                    image_norm = (image_resized - np.min(image_resized))/(np.max(image_resized) - np.min(image_resized))
#                     image_norm = (image_resized - image_resized.mean())/ image_resized.std()


                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    # normalize each channel before loading into container???????????????????
                    
#                     batch_data[folder,idx,:,:,0] = image_norm[:, :, 0]
                    batch_data[folder,idx,:,:,0] = (image_resized[:, :, 0] - np.min(image_resized[:, :, 0]))/(np.max(image_resized[:, :, 0]) - np.min(image_resized[:, :, 0]))
#                     batch_data[folder,idx,:,:,1] = image_norm[:, :, 1]
                    batch_data[folder,idx,:,:,1] = (image_resized[:, :, 1] - np.min(image_resized[:, :, 1]))/(np.max(image_resized[:, :, 1]) - np.min(image_resized[:, :, 1]))
#                     batch_data[folder,idx,:,:,2] = image_norm[:, :, 2]
                    batch_data[folder,idx,:,:,2] = (image_resized[:, :, 2] - np.min(image_resized[:, :, 2]))/(np.max(image_resized[:, :, 2]) - np.min(image_resized[:, :, 2]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
#             return batch_data, batch_labels

        # write the code for the remaining data points which are left after full batches
        batch = num_batches

#         folders_covered_size = num_batches * batch_size
#         folders_remain_size = len(t) - folders_covered_size
        folders_remain_size = len(t) % batch_size
        

        batch_data = np.zeros((folders_remain_size, 12, 100, 100, 3)) 
        batch_labels = np.zeros((folders_remain_size,5))
        
        for folder in range(folders_remain_size):
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = io.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]) #.astype(np.float32)
                    
                image_resized = skimage.transform.resize(image, (100, 100))#, anti_aliasing=True)
                    
                image_norm = (image_resized - np.min(image_resized))/(np.max(image_resized) - np.min(image_resized))
                    
#                 batch_data[folder,idx,:,:,0] = image_norm[:, :, 0]
                batch_data[folder,idx,:,:,0] = (image_resized[:, :, 0] - np.min(image_resized[:, :, 0]))/(np.max(image_resized[:, :, 0]) - np.min(image_resized[:, :, 0]))
#                 batch_data[folder,idx,:,:,1] = image_norm[:, :, 1]
                batch_data[folder,idx,:,:,1] = (image_resized[:, :, 1] - np.min(image_resized[:, :, 1]))/(np.max(image_resized[:, :, 1]) - np.min(image_resized[:, :, 1]))
#                 batch_data[folder,idx,:,:,2] = image_norm[:, :, 2]
                batch_data[folder,idx,:,:,2] = (image_resized[:, :, 2] - np.min(image_resized[:, :, 2]))/(np.max(image_resized[:, :, 2]) - np.min(image_resized[:, :, 2]))
                    
            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

        yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [0]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/Project_data/train'
val_path = '/content/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
#num_epochs = # choose the number of epochs
num_epochs = 25
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 25


In [0]:
# Useful
#Input shape

#5D tensor with shape: (batch, channels, conv_dim1, conv_dim2, conv_dim3) if data_format is  "channels_first" or 
#5D tensor with shape: (batch, conv_dim1, conv_dim2, conv_dim3, channels) if  data_format is "channels_last".



In [0]:
print(batch_data.shape[1:][1:])

(60, 60, 3)


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [0]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras import regularizers

#write your model here
model = Sequential()
model.add(Conv3D(32, (3, 3, 3), padding='same', input_shape=(12,100,100,3)))
model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(Conv3D(32, (3, 3, 3), padding='same'))
model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2,2,2)))
#model.add(Dropout(0.25))

model.add(Conv3D(64, (3, 3, 3), padding='same'))
model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(Conv3D(64, (3, 3, 3), padding='same'))
model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2,2,2)))
#model.add(Dropout(0.25))

model.add(Conv3D(128, (3, 3, 3), padding='same'))
model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(Conv3D(128, (3, 3, 3), padding = 'same'))
model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2,2,2)))
#model.add(Dropout(0.25))

model.add(Conv3D(256, (3, 3, 3), padding='same'))
model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(Conv3D(256, (3, 3, 3), padding = 'same'))
model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(2,2,2)))
#model.add(Dropout(0.25))

model.add(Conv3D(512, (3, 3, 3), padding='same'))
model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(2,2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

#model.add(Dense(1024, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1024))
model.add(Activation('relu'))

model.add(Dropout(0.5))

num_classes = 5
model.add(Dense(num_classes, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [0]:
#optimiser = #write your optimizer
#optimiser = 'sgd'
optimiser = optimizers.SGD(lr=0.001)
#optimiser = optimizers.SGD(lr=0.001,decay=1e-6, momentum=0.9, nesterov=True)
#optimiser = 'adagrad'
#optimiser = 'adam'
model.compile(optimizer = optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 12, 100, 100, 32)  2624      
_________________________________________________________________
activation_1 (Activation)    (None, 12, 100, 100, 32)  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 12, 100, 100, 32)  128       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 12, 100, 100, 32)  27680     
_________________________________________________________________
activation_2 (Activation)    (None, 12, 100, 100, 32)  0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 12, 100, 100, 32)  128       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 6, 50, 50, 32)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [0]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)


In [0]:
print((train_doc[:12]))

['WIN_20180925_18_17_43_Pro_Right_Swipe_new;Right_Swipe_new;1\n'
 'WIN_20180926_17_23_39_Pro_Left_Swipe_new;Left_Swipe_new;0\n'
 'WIN_20180925_17_50_44_Pro_Left_Swipe_new;Left_Swipe_new;0\n'
 'WIN_20180926_16_50_28_Pro_Right_Swipe_new;Right_Swipe_new;1\n'
 'WIN_20180925_17_48_48_Pro_Right_Swipe_new;Right_Swipe_new;1\n'
 'WIN_20180925_17_49_07_Pro_Left_Swipe_new;Left_Swipe_new;0\n'
 'WIN_20180925_17_50_31_Pro_Right_Swipe_new;Right_Swipe_new;1\n'
 'WIN_20180925_18_16_51_Pro_Right_Swipe_new;Right_Swipe_new;1\n'
 'WIN_20180925_17_52_46_Pro_Right_Swipe_new;Right_Swipe_new;1\n'
 'WIN_20180926_17_20_39_Pro_Right_Swipe_new;Right_Swipe_new;1\n'
 'WIN_20180926_17_16_10_Pro_Left_Swipe_new;Left_Swipe_new;0\n'
 'WIN_20180925_17_18_25_Pro_Right_Swipe_new;Right_Swipe_new;1\n']


In [0]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

#LR = # write the REducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0001)
callbacks_list = [checkpoint, LR]
#callbacks_list = [checkpoint]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [0]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
print("Steps per Epoch: ", steps_per_epoch)
print("Validation Steps: ", validation_steps)

Steps per Epoch:  23
Validation Steps:  4


Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [0]:
#model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                    callbacks=callbacks_list, validation_data=val_generator, 
#                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)


Source path =  /content/Project_data/train ; batch size = 30
Epoch 1/25


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


23/23 [==============================] - 108s 5s/step - loss: 2.8681 - categorical_accuracy: 0.3449 - val_loss: 3.2724 - val_categorical_accuracy: 0.3300

Epoch 00001: saving model to model_init_2018-10-1917_13_20.278326/model-00001-2.86708-0.34540-3.27244-0.33000.h5
Epoch 2/25
23/23 [==============================] - 84s 4s/step - loss: 2.1597 - categorical_accuracy: 0.4001 - val_loss: 3.3662 - val_categorical_accuracy: 0.4200

Epoch 00002: saving model to model_init_2018-10-1917_13_20.278326/model-00002-2.09733-0.40271-3.36617-0.42000.h5
Epoch 3/25
23/23 [==============================] - 85s 4s/step - loss: 1.7335 - categorical_accuracy: 0.4887 - val_loss: 2.7911 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to model_init_2018-10-1917_13_20.278326/model-00003-1.69735-0.49472-2.79110-0.50000.h5
Epoch 4/25
23/23 [==============================] - 81s 4s/step - loss: 1.5489 - categorical_accuracy: 0.5317 - val_loss: 1.2966 - val_categorical_accuracy: 0.5400

Epoch 00004

In [0]:
# The link to access the model file on the drive is given below:

https://drive.google.com/open?id=1Fl5ZmCnHjefuEopZz00TJ4H1hQBndh3z